In [1]:
import numpy as np
import pandas as pd
from pyscf import gto, scf
import ext 

2025-04-18 21:31:56.318888: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 21:31:56.328918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745029916.341175    8490 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745029916.344763    8490 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 21:31:56.357016: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
from connect import conn
engine = conn()
from os import chdir
chdir('..')
import sopy as sp

In [3]:
def D(q, method):
    return pd.concat([Run,pd.Series({'method':method, 'dist':(q.dist(Sum1)).numpy(), 'canons':int(len(q)), 'ld2':q.ld2().numpy(), 'ld1':q.ld1().numpy(),'norm':q.n().numpy()})])


In [4]:
def rotation_matrix_from_euler(roll, pitch, yaw, vector):
    """
    Creates a 3D rotation matrix from Euler angles (roll, pitch, yaw) in radians.

    Args:
        roll: Rotation around the X-axis (radians).
        pitch: Rotation around the Y-axis (radians).
        yaw: Rotation around the Z-axis (radians).

    Returns:
        A 3x3 NumPy rotation matrix.

    generated by Gemini
    """
    cos_r, sin_r = np.cos(roll), np.sin(roll)
    cos_p, sin_p = np.cos(pitch), np.sin(pitch)
    cos_y, sin_y = np.cos(yaw), np.sin(yaw)

    rx = np.array([[1, 0, 0],
                   [0, cos_r, -sin_r],
                   [0, sin_r, cos_r]])

    ry = np.array([[cos_p, 0, sin_p],
                   [0, 1, 0],
                   [-sin_p, 0, cos_p]])

    rz = np.array([[cos_y, -sin_y, 0],
                   [sin_y, cos_y, 0],
                   [0, 0, 1]])
    def p(v):
        return (f" {v[0]} {v[1]} {v[2]} ")
    return p(rz @ ry @ rx @vector) # Order: ZYX (yaw, pitch, roll)

# Example Usage:
roll, pitch, yaw = 0*np.radians(np.pi/4), 0*np.radians(np.pi/2), 0*np.radians(-np.pi/4)
rotation_matrix_from_euler(roll, pitch, yaw, [2,2,2])

Run = pd.Series([ roll, pitch, yaw] , index = ['roll','pitch','yaw'])
Run

roll     0.0
pitch    0.0
yaw     -0.0
dtype: float64

In [5]:
mol = gto.Mole()

if False:
    mol.build( atom = '''H 0 0 1;  ''', basis ={'H': [[0,[np.power(4.,n),1]] for n in range (-2,4)]},spin= 1)

    ne = 1
    canons = 1

    filename = 'H'
if True:

    
    mol.build( atom = f'''H {rotation_matrix_from_euler(roll, pitch, yaw, [2,2,2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [2,-2,-2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [-2,2,-2])}; H {rotation_matrix_from_euler(roll, pitch, yaw, [-2,-2,2])} ''', basis ={'H': [[0,[np.power(4.,n),1]] for n in range (-2,4)]},spin= 0)
    
    filename = 'H4'

    ne = 4
    canons = ne // 2
    
##

if False:
    coord =   """6.290730e-01,6.290730e-01,6.290730e-01
      6.290730e-01,-6.290730e-01 ,-6.290730e-01
     -6.290730e-01, 6.290730e-01 ,-6.290730e-01
     -6.290730e-01,-6.290730e-01 , 6.290730e-01
      3.629073e+00, 6.290730e-01 , 6.290730e-01
      3.629073e+00,-6.290730e-01 ,-6.290730e-01
      2.370927e+00, 6.290730e-01 ,-6.290730e-01
      2.370927e+00,-6.290730e-01 , 6.290730e-01
      6.290730e-01, 3.629073e+00 , 6.290730e-01
     6.290730e-01,  2.370927e+00 ,-6.290730e-01
     -6.290730e-01,  3.629073e+00 ,-6.290730e-01
     -6.290730e-01,  2.370927e+00 , 6.290730e-01
     6.290730e-01,  6.290730e-01 , 3.629073e+00
     6.290730e-01, -6.290730e-01 , 2.370927e+00
     -6.290730e-01,  6.290730e-01,  2.370927e+00
     -6.290730e-01, -6.290730e-01,  3.629073e+00"""
    coords = ''
    for line in coord.split('\n'):
        x,y,z = map(float,line.replace(' ','').split(','))
        coords += f' H {rotation_matrix_from_euler(roll, pitch, yaw, [x,y,z])} ; '
    mol.build( atom = coords, basis ={'H': [[0,[np.power(4.,n),1]] for n in range (-2,4)]},spin= 0)
    ne = 16
    filename = 'H16'
    canons = ne *4
# if False:
#     mol.build( atom = f'''
#     C {rotation_matrix_from_euler(roll, pitch, yaw, [1.390 0.000 0.000])},
#     C {rotation_matrix_from_euler(roll, pitch, yaw, [0.695 1.202 0.000])},
#     C {rotation_matrix_from_euler(roll, pitch, yaw, [-0.695 1.202 0.000])},
#     C {rotation_matrix_from_euler(roll, pitch, yaw, [-1.390 0.000 0.000])},
#     C {rotation_matrix_from_euler(roll, pitch, yaw, [-0.695 -1.202 0.000])},
#     C {rotation_matrix_from_euler(roll, pitch, yaw, [0.695 -1.202 0.000])},
#     H {rotation_matrix_from_euler(roll, pitch, yaw, [2.470 0.000 0.000])},
#     H {rotation_matrix_from_euler(roll, pitch, yaw, [1.235 2.140 0.000])},
#     H {rotation_matrix_from_euler(roll, pitch, yaw, [-1.235 2.140 0.000])},
#     H {rotation_matrix_from_euler(roll, pitch, yaw, [-2.470 0.000 0.000])},
#     H {rotation_matrix_from_euler(roll, pitch, yaw, [-1.235 -2.140 0.000])},
#     H {rotation_matrix_from_euler(roll, pitch, yaw, [1.235 -2.140 0.000])}
#  ''', basis ={'H': [[0,[np.power(4.,n),1]] for n in range (-2,4)],'C': [[0,[np.power(4.,n),1]] for n in range (-2,4)]+[[1,[np.power(4.,n),1]] for n in range (-1,2)]},spin= 0)

#     ne = 42
#     canons = 16
#     filename = 'benzene'
# ##


Run['ne']     = ne
Run['filename'] = filename

In [6]:
def deploy ( basis ):
    n= {0:1, 1:3, 2:6, 3:10, 4:15}
    stage = []
    for ii,(l,(exp,coef)) in enumerate(basis):
        for iii in range( n[l] ):
            stage += [exp]
    return (stage)

In [7]:
labels = mol.cart_labels()
at = [ str(labels[i][2:5]).strip() for i in range(len(labels))]
ii = [ int(str(labels[i][:2].replace(' ',''))) for i in range(len(labels))] 
ll = [[ (labels[i][7:]).count(axis)  for axis in ['x','y','z']] for i in range(len(labels))]
xx = [ [mol.atom_coord(ii[i])[x] for x in [0,1,2]] for i in range(len(labels)) ]
Data = pd.DataFrame(np.transpose([at,ii,np.transpose(ll)[0],np.transpose(ll)[1],np.transpose(ll)[2],np.transpose(xx)[0], np.transpose(xx)[1], np.transpose(xx)[2]]), columns = ['ATOM','#','lx','ly','lz','x','y','z'])
Data['exp'] = np.nan
Data['l'] = Data[['lx','ly','lz']].astype(int).sum(1)
Data = Data.set_index(['ATOM','#'])
for atom, num in Data.index.unique():
    Data.loc[(atom,num),'exp'] = deploy( mol.basis[atom] )

Data[['lx','x','ly','y','lz','z','exp']].shape


(24, 7)

In [8]:
## RHF ##
##     ##
mf = scf.RHF(mol).run()
orb = mf.mo_coeff ##First Level##
s = mol.intor('int1e_ovlp')

##molecular orbital in terms of basis elements

converged SCF energy = -1.5265417303906


In [9]:
Data = Data[['lx','x','ly','y','lz','z','exp']]
for elec in range(mol.nelectron):
    Data[elec+1] = orb[:,elec]
Data = Data.reset_index()


In [10]:
box_size = 50.0
L        = 500
Run['box_size'] = box_size
Run['L']        = L


lattices = 3*[ np.linspace( -box_size/2,box_size/2, L ) ]
Sum = sp.vector()
for index in Data.index:
    positions = Data.loc[index,['x','y','z']].astype(float).values
    ls        = Data.loc[index,['lx','ly','lz']].astype(float).values
    sigmas    = Data.loc[index,['exp','exp','exp']].astype(float).values
    v = sp.vector().gaussian(ls=ls,positions=positions, sigmas=sigmas, a = Data.loc[index,ne], lattices = lattices)
    Sum += v

Sum1 = Sum.mul(1./Sum.n())
np.sqrt(Sum.dot(Sum)), len(Sum),Sum.ld1(), Sum.ld2()
np.sqrt(Sum1.dot(Sum1)), len(Sum1),Sum1.ld1(), Sum1.ld2()

2025-04-18 21:32:00.727325: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


(np.float64(1.0),
 24,
 <tf.Tensor: shape=(), dtype=float64, numpy=2.0676820910937774>,
 <tf.Tensor: shape=(), dtype=float64, numpy=0.831325538619632>)

In [11]:
canons

2

In [12]:
q = Sum1.fibonacci(  partition = canons, iterate = 10,total_iterate = 100 ,alpha = 1e-6)
q3 = Sum1.boost().decompose(  partition = canons, iterate = 10 ,alpha = 1e-6).unboost()
q2 = Sum1.boost().fibonacci(  partition = canons, iterate = 10,total_iterate = 100 ,alpha = 1e-6).unboost()

In [13]:
Tum1 = { 0: Sum1[0], 1: Sum1[1], 2: Sum1[2], 3: Sum1[3] } 

In [14]:
tl = ext.reduce(ext.image(Tum1), canons )

tl1 = sp.vector().transpose(tl)
D(tl1,'reduce')

roll             0.0
pitch            0.0
yaw             -0.0
ne               4.0
filename          H4
box_size        50.0
L                500
method        reduce
dist        0.373083
canons             2
ld2         1.414214
ld1              2.0
norm        0.927798
dtype: object

In [15]:
(pd.DataFrame([D(Sum1,'Orbital'),D(q,'fibonacci'),D(q2,'boost_fibonacci'),D(tl1,'reduce')])).to_sql('pySCF',engine, if_exists = 'append', index= False)

4